In [1]:
!python3 -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.3.0) =================
ℹ spaCy installation: /opt/conda/lib/python3.7/site-packages/spacy

NAME              SPACY                 VERSION                            
en_core_web_sm    >=3.3.0.dev0,<3.4.0   3.3.0   ✔
en_core_web_trf   >=3.3.0.dev0,<3.4.0   3.3.0   ✔



In [2]:
import os
import re
import pandas as pd

### Dir Locations

In [4]:
!ls /path/to/dir/spacy_model_training_ner/data/diease_ner/train_dev_test_split_conll_data/

dev_data.conll	test_data.conll  train_data.conll


In [5]:
DATA_DIR = "/path/to/dir/spacy_model_training_ner/data/diease_ner/train_dev_test_split_conll_data/"
SPACY_DATA_DIR = "/path/to/dir/spacy_model_training_ner/data/diease_ner/train_dev_test_split_spacy_binary/"
CONFIG_DIR = "/path/to/dir/spacy_model_training_ner/data/model_config/spacy_small_model/" 
SPACY_SMALL_MODEL_DIR_GPU = '/path/to/dir/spacy_model_training_ner/data/model_weights/spacy_small/'

In [6]:
!mkdir -p $SPACY_DATA_DIR $CONFIG_DIR $SPACY_SMALL_MODEL_DIR_GPU

### 1. Spacy Convert
- conll to spacy compatible format

In [7]:
!python3 -m spacy convert $DATA_DIR/train_data.conll $SPACY_DATA_DIR -c ner -n 1
!python3 -m spacy convert $DATA_DIR/dev_data.conll $SPACY_DATA_DIR -c ner -n 1
!python3 -m spacy convert $DATA_DIR/test_data.conll $SPACY_DATA_DIR -c iob -n 1

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (5836 documents):
/path/to/dir/spacy_model_training_ner/data/diease_ner/train_dev_test_split_spacy_binary/train_data.spacy
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (729 documents):
/path/to/dir/spacy_model_training_ner/data/diease_ner/train_dev_test_split_spacy_binary/dev_data.spacy
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (730 documents):
/path/to/dir/spacy_model_training_ner/data/diease_ner/train_dev_test_split_spacy_binary/test_data.spacy


### 2. Spacy Init Config

In [8]:
!python3 -m spacy init config $CONFIG_DIR/original_spacy_small_ner_config.cfg --lang en --pipeline "ner" --optimize efficiency --force

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
/path/to/dir/spacy_model_training_ner/data/model_config/spacy_small_model/original_spacy_small_ner_config.cfg
You can now add your data and train your pipeline:
python -m spacy train original_spacy_small_ner_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
!cat $CONFIG_DIR/original_spacy_small_ner_config.cfg

[paths]
train = null
dev = null
vectors = null
init_tok2vec = null

[system]
gpu_allocator = null
seed = 0

[nlp]
lang = "en"
pipeline = ["tok2vec","ner"]
batch_size = 1000
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}
upstream = "*"

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = ${compon

**Changes made to the config**: <br> 
- the dropout from 0.1 to 0.3 to make memorizing of the Rules NER output difficult
<!-- - Added pipeline components like tagger, parser (needed for subsequent Rules NER) 
    - Also add them to frozen_components = ["tagger","parser"] -->
- Some important training pieces
```
patience = 3000
max_epochs = 50
max_steps = 20000
```
- Changed Console logger `progress_bar` to true

In [11]:
%%writefile $CONFIG_DIR/original_spacy_small_ner_config.cfg
[paths]
train = null
dev = null
vectors = null
init_tok2vec = null

[system]
gpu_allocator = null
seed = 0

[nlp]
lang = "en"
pipeline = ["tok2vec","ner"]
batch_size = 1000
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}
upstream = "*"

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = ${components.tok2vec.model.encode.width}
attrs = ["NORM","PREFIX","SUFFIX","SHAPE"]
rows = [5000,2500,2500,2500]
include_static_vectors = false

[components.tok2vec.model.encode]
@architectures = "spacy.MaxoutWindowEncoder.v2"
width = 96
depth = 4
window_size = 1
maxout_pieces = 3

[corpora]

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = ${paths.dev}
max_length = 0
gold_preproc = false
limit = 0
augmenter = null

[corpora.train]
@readers = "spacy.Corpus.v1"
path = ${paths.train}
max_length = 0
gold_preproc = false
limit = 0
augmenter = null

[training]
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"
seed = ${system.seed}
gpu_allocator = ${system.gpu_allocator}
dropout = 0.3
accumulate_gradient = 1
patience = 3000
max_epochs = 50
max_steps = 20000
eval_frequency = 200
frozen_components = []
annotating_components = []
before_to_disk = null

[training.batcher]
@batchers = "spacy.batch_by_words.v1"
discard_oversize = false
tolerance = 0.2
get_length = null

[training.batcher.size]
@schedules = "compounding.v1"
start = 100
stop = 1000
compound = 1.001
t = 0.0

[training.logger]
@loggers = "spacy.ConsoleLogger.v1"
progress_bar = true

[training.optimizer]
@optimizers = "Adam.v1"
beta1 = 0.9
beta2 = 0.999
L2_is_weight_decay = true
L2 = 0.01
grad_clip = 1.0
use_averages = false
eps = 0.00000001
learn_rate = 0.001

[training.score_weights]
ents_f = 1.0
ents_p = 0.0
ents_r = 0.0
ents_per_type = null

[pretraining]

[initialize]
vectors = ${paths.vectors}
init_tok2vec = ${paths.init_tok2vec}
vocab_data = null
lookups = null
before_init = null
after_init = null

[initialize.components]

[initialize.tokenizer]

Overwriting /path/to/dir/spacy_model_training_ner/data/model_config/spacy_small_model//original_spacy_small_ner_config.cfg


In [12]:
from configparser import ConfigParser
configur = ConfigParser()
configur.read(f'{CONFIG_DIR}/original_spacy_small_ner_config.cfg')
print(configur.sections())

['paths', 'system', 'nlp', 'components', 'components.ner', 'components.ner.model', 'components.ner.model.tok2vec', 'components.tok2vec', 'components.tok2vec.model', 'components.tok2vec.model.embed', 'components.tok2vec.model.encode', 'corpora', 'corpora.dev', 'corpora.train', 'training', 'training.batcher', 'training.batcher.size', 'training.logger', 'training.optimizer', 'training.score_weights', 'pretraining', 'initialize', 'initialize.components', 'initialize.tokenizer']


In [13]:
# Type of NER model
dict(configur.items("components.ner.model"))

{'@architectures': '"spacy.TransitionBasedParser.v2"',
 'state_type': '"ner"',
 'extra_state_tokens': 'false',
 'hidden_width': '64',
 'maxout_pieces': '2',
 'use_upper': 'true',
 'no': 'null'}

In [14]:
# critical config changes
!head -n 60 $CONFIG_DIR/original_spacy_small_ner_config.cfg | tail -n 50

[nlp]
lang = "en"
pipeline = ["tok2vec","ner"]
batch_size = 1000
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}
upstream = "*"

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = ${components.tok2vec.model.encode.width}
attrs = ["NORM","PREFIX","SUFFIX","SHAPE"]
rows = [5000,2500,2500,2500]
inc

### 4. Debug Spacy Data

In [16]:
!python3 -m spacy debug data $CONFIG_DIR/original_spacy_small_ner_config.cfg \
--paths.train $SPACY_DATA_DIR/train_data.spacy \
--paths.dev $SPACY_DATA_DIR/dev_data.spacy \
--paths.test $SPACY_DATA_DIR/test_data.spacy \
--verbose


============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
5836 training docs
729 evaluation docs
⚠ 5 training examples also in evaluation data

============================== Vocab & Vectors ==============================
ℹ 148187 total word(s) in the data (9829 unique)
10 most common words: '.' (6708), 'the' (6121), 'of' (5842), ',' (5117), 'in'
(4022), '-' (3911), 'and' (3471), 'a' (2532), ')' (2093), '(' (2084)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 2 label(s)
0 missing value(s) (tokens with '-' label)
Labels in train data: 'EP]', 'DISEASE'
⚠ Low number of examples for label 'EP]' (7)
✔ Examples without occurrences available for all labels
✔ No entities consisting of or starting/ending with white

### 5. Train Custom NER Model

In [17]:
%%time
!python3 -m spacy train $CONFIG_DIR/original_spacy_small_ner_config.cfg \
--output $SPACY_SMALL_MODEL_DIR_GPU \
--paths.train $SPACY_DATA_DIR/train_data.spacy \
--paths.dev $SPACY_DATA_DIR/dev_data.spacy \
--verbose \
-g 0

[2022-05-24 07:09:39,410] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
ℹ Saving to output directory:
/path/to/dir/SpacyNER/data/model_weights/spacy_small
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-05-24 07:09:41,789] [INFO] Set up nlp object from config
[2022-05-24 07:09:41,797] [DEBUG] Loading corpus from path: /path/to/dir/spacy_model_training_ner/data/diease_ner/train_dev_test_split_spacy_binary/dev_data.spacy
[2022-05-24 07:09:41,798] [DEBUG] Loading corpus from path: /path/to/dir/spacy_model_training_ner/data/diease_ner/train_dev_test_split_spacy_binary/train_data.spacy
[2022-05-24 07:09:41,798] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-24 07:09:41,803] [INFO] Created vocabulary
[2022-05-24 07:09:41,804] [INFO] Finished initializing nlp object
[2022-05-24 07:09:46,631] [DEBUG] [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the mo